In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import pickle
import math

In [2]:
models = ["SVM", "CNN", "GRU", "BERT", "MAXi", "MAXh", "ATTh", "MAX", "ATT"]

In [3]:
df = {}

In [4]:
dfSiteCNN = pd.read_csv("f1matrix-CNN-site.csv")
df["site"] = pd.read_csv("f1matrix-site.csv")
df["site"]["CNN"] = dfSiteCNN["CNN"]

In [5]:
dfMorphoCNN = pd.read_csv("f1matrix-CNN-morpho.csv")
df["morpho"] = pd.read_csv("f1matrix-morpho.csv")
df["morpho"]["CNN"] = dfMorphoCNN["CNN"]

In [6]:
df["site"]

,icdo,GRU,MAXi,MAXh,ATTh,ATT,MAX,SVM,XGBoost,BERT,examples,CNN
0,0,0.565217,0.000000,0.619048,0.500000,0.666667,0.564103,0.370370,0.451613,0.473684,18,0.437500
1,1,0.285714,0.000000,0.200000,0.428571,0.300000,0.500000,0.222222,0.000000,0.000000,7,0.307692
2,2,0.736842,0.408163,0.724138,0.684932,0.754098,0.828571,0.794118,0.794118,0.777778,34,0.686567
3,3,0.250000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,5,0.000000
4,4,0.000000,0.000000,0.000000,0.222222,0.000000,0.250000,0.571429,0.571429,0.000000,7,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
63,74,0.500000,0.500000,0.500000,0.500000,0.400000,0.666667,0.666667,0.666667,0.500000,2,0.666667
64,75,0.820513,0.810811,0.789474,0.638298,0.727273,0.714286,0.842105,0.777778,0.777778,21,0.756757
65,76,0.000000,0.000000,0.000000,0.000000,0.400000,0.000000,0.000000,0.000000,0.000000,3,0.000000
66,77,0.602190,0.360743,0.645051,0.650264,0.660650,0.662116,0.384824,0.290909,0.664234,264,0.637324


In [7]:
df["morpho"]

,icdo,GRU,MAXi,MAXh,ATTh,ATT,MAX,SVM,XGBoost,BERT,examples,CNN
0,8000,0.375185,0.295875,0.383333,0.324159,0.409704,0.430052,0.279612,0.397790,0.446429,345,0.240000
1,8001,0.076923,0.000000,0.075000,0.186916,0.114943,0.144578,0.095238,0.075949,0.000000,75,0.051948
2,8010,0.542735,0.439174,0.535044,0.561257,0.584388,0.562173,0.509485,0.628722,0.618009,561,0.501403
3,8011,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,0.000000
4,8012,0.500000,0.000000,0.363636,0.727273,0.857143,0.400000,0.400000,0.500000,0.571429,4,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...
198,9980,0.463768,0.307692,0.470588,0.418605,0.578947,0.555556,0.478261,0.444444,0.327273,22,0.436364
199,9983,0.200000,0.222222,0.307692,0.333333,0.461538,0.363636,0.400000,0.461538,0.000000,8,0.400000
200,9984,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,0.000000
201,9989,0.426667,0.573171,0.470588,0.493506,0.530612,0.574850,0.554745,0.535948,0.444444,78,0.451613


# Tables 1 & 2

In [8]:
toRemove = {
    "site": pickle.load(open("toRemoveSite.p", 'rb')),
    "morpho": pickle.load(open("toRemoveMorfo.p", 'rb')),
}

In [9]:
icdo = {"site":[], "morpho":[]}
samples = {"site":[], "morpho":[]}
f1 = {"site":{}, "morpho":{}}
for m in models:
    f1["site"][m] = []
    f1["morpho"][m] = []

In [10]:
for t in df.keys():
    for i in range(len(df[t])):
        if not df[t].icdo[i] in toRemove[t]:
            icdo[t].append(list(df[t].icdo)[i])
            samples[t].append(list(df[t].examples)[i])
            for m in models:
                f1[t][m].append(list(df[t][m])[i])

In [11]:
d = {"site": {}, "morpho": {}}
for t in d.keys():
    for m in [m for m in models if m != "MAX"]:
        d[t][m] = []

In [12]:
for t in d:
    for m in d[t]:
        for i in range(len(icdo[t])):
            d[t][m].append(f1[t]["MAX"][i] - f1[t][m][i])

In [13]:
dAvg = {}
for t in d:
    dAvg[t] = {}
    for m in d[t]:
        dAvg[t][m] = sum(d[t][m])/len(d[t][m])

In [14]:
dS = {}
for t in d:
    dS[t] = {}
    for m in d[t]:
        dS[t][m] = []
        for i in range(len(d[t][m])):
            dS[t][m].append((d[t][m][i] - dAvg[t][m]) **2)

In [15]:
T = {}
for t in dAvg:
    T[t] = {}
    for m in dAvg[t]:
        T[t][m] = dAvg[t][m]/(math.sqrt(sum(dS[t][m])/(len(dS[t][m])))/math.sqrt(len(dS[t][m])))
        #T[t][m] = dAvg[t][m]/(math.sqrt(sum(dS[t][m])/(len(dS[t][m])-1))/math.sqrt(len(dS[t][m])))

In [16]:
p = {}
for t in T:
    p[t] = {}
    for m in T[t]:
        n = sum([1 for i in range(len(f1[t][m])) if f1[t][m][i] != f1[t]["MAX"][i]])
        if n <= 40:
            p[t][m] = 1 - stats.t.cdf(T[t][m],df=n-1)
        else:
            p[t][m] = 1 - stats.norm.cdf(T[t][m])

In [17]:
p

{'site': {'SVM': 0.0729571367676346,
  'CNN': 3.856254315870444e-08,
  'GRU': 0.0007832276579321507,
  'BERT': 0.007414466371396533,
  'MAXi': 6.62325042588563e-08,
  'MAXh': 0.003477946891744854,
  'ATTh': 0.0005666687233821799,
  'ATT': 0.08336492222972736},
 'morpho': {'SVM': 0.0004993641544911442,
  'CNN': 0.00027746709458020824,
  'GRU': 0.00013042514055749965,
  'BERT': 1.6435670870174057e-05,
  'MAXi': 0.0,
  'MAXh': 0.002503453302247882,
  'ATTh': 0.18166954331335172,
  'ATT': 0.9867910845390301}}

# Table 3

In [18]:
low = 5 # ignore below this
mid = 100 # upper bound of the easy class
high = 1000 # upper bount of the avg class

In [19]:
df3 = {}
for t in df:
    df3[t] = {
        "hard": df[t].loc[(df[t]['examples'] >= low) & (df[t]['examples'] < mid)],
        "avg": df[t].loc[(df[t]['examples'] >= mid) & (df[t]['examples'] < high)],
        "easy": df[t].loc[df[t]['examples'] >= high],
    }

In [20]:
icdo3 = {"site":[], "morpho":[]}
samples3 = {"site":[], "morpho":[]}
f13 = {"site":{}, "morpho":{}}
for t in df3:
    icdo3[t] = {}
    samples3[t] = {}
    f13[t] = {}
    for c in df3[t]:
        icdo3[t][c] = []
        samples3[t][c] = []
        f13[t][c] = {}
        for m in models:
            f13[t][c][m] = []

In [21]:
for t in df3:
    for c in df3[t]:
        for i in range(len(df3[t][c])):
            icdo3[t][c].append(list(df3[t][c].icdo)[i])
            samples3[t][c].append(list(df3[t][c].examples)[i])
            for m in models:
                f13[t][c][m].append(list(df3[t][c][m])[i])

In [22]:
d3 = {}
for t in f13:
    d3[t] = {}
    for c in f13[t]:
        d3[t][c] = {}
        for m in [m for m in models if m != "MAX"]:
            d3[t][c][m] = []

In [23]:
for t in d3:
    for c in d3[t]:
        for m in d3[t][c]:
            for i in range(len(icdo3[t][c])):
                d3[t][c][m].append(f13[t][c]["MAX"][i] - f13[t][c][m][i])

In [24]:
dAvg3 = {}
for t in d3:
    dAvg3[t] = {}
    for c in d3[t]:
        dAvg3[t][c] = {}
        for m in d3[t][c]:
            dAvg3[t][c][m] = sum(d3[t][c][m])/len(d3[t][c][m])

In [25]:
dS3 = {}
for t in d3:
    dS3[t] = {}
    for c in d3[t]:
        dS3[t][c] = {}
        for m in d3[t][c]:
            dS3[t][c][m] = []
            for i in range(len(d3[t][c][m])):
                dS3[t][c][m].append((d3[t][c][m][i] - dAvg3[t][c][m]) **2)

In [26]:
T3 = {}
for t in dAvg3:
    T3[t] = {}
    for c in dAvg3[t]:
        T3[t][c] = {}
        for m in dAvg3[t][c]:
            T3[t][c][m] = dAvg3[t][c][m]/(math.sqrt(sum(dS3[t][c][m])/(len(dS3[t][c][m])))/math.sqrt(len(dS3[t][c][m])))
            #T3[t][c][m] = dAvg3[t][c][m]/(math.sqrt(sum(dS3[t][c][m])/(len(dS3[t][c][m])-1))/math.sqrt(len(dS3[t][c][m])))

In [27]:
p3 = {}
for t in T3:
    p3[t] = {}
    for c in T3[t]:
        p3[t][c] = {}
        for m in T3[t][c]:
            n = sum([1 for i in range(len(f13[t][c][m])) if f13[t][c][m][i] != f13[t][c]["MAX"][i]])
            if n <= 40:
                p3[t][c][m] = 1 - stats.t.cdf(T3[t][c][m],df=n-1)
            else:
                p3[t][c][m] = 1 - stats.norm.cdf(T3[t][c][m])

In [28]:
p3

{'site': {'hard': {'SVM': 0.1442209054531406,
   'CNN': 4.691033652859389e-06,
   'GRU': 0.0026278340067553696,
   'BERT': 0.009246724674393803,
   'MAXi': 7.244155835750554e-06,
   'MAXh': 0.008632365187028479,
   'ATTh': 0.0009682265656497657,
   'ATT': 0.09468457441737343},
  'avg': {'SVM': 0.11072887510253382,
   'CNN': 0.0008274256427013293,
   'GRU': 0.040496665312818814,
   'BERT': 0.3143026799817108,
   'MAXi': 0.017817775398876634,
   'MAXh': 0.05662220928894601,
   'ATTh': 0.33490307586567036,
   'ATT': 0.24198506001129405},
  'easy': {'SVM': 0.003305989965765699,
   'CNN': 0.02989965250212223,
   'GRU': 0.7713588558466036,
   'BERT': 0.04536089796558851,
   'MAXi': 0.055573803157068946,
   'MAXh': 0.02304264532484268,
   'ATTh': 0.6043047554989713,
   'ATT': 0.46538116730846646}},
 'morpho': {'hard': {'SVM': 0.0015340833212037497,
   'CNN': 0.0005279136898266312,
   'GRU': 0.0001892027082712744,
   'BERT': 4.95233407626916e-06,
   'MAXi': 0.0,
   'MAXh': 0.002658780021907447